In [28]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
import os
from transformers import BertForQuestionAnswering, BertTokenizer, BartForConditionalGeneration, DPRContextEncoder, DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer

In [7]:
# Load the data
paragraphs_df = pd.read_excel('paragraphs.xlsx')
queries_df = pd.read_excel('queries.xlsx')
print(paragraphs_df.head())
print(queries_df.head())

     25909  Zhejiang  \
0  25912.0  Zhejiang   
1  25913.0  Zhejiang   
2  25914.0  Zhejiang   
3  25915.0  Zhejiang   
4  25916.0  Zhejiang   

   Zhejiang (help·info), formerly romanized as Chekiang, is an eastern coastal province of China. Zhejiang is bordered by Jiangsu province and Shanghai municipality to the north, Anhui province to the northwest, Jiangxi province to the west, and Fujian province to the south; to the east is the East China Sea, beyond which lie the Ryukyu Islands of Japan.  
0   Zhejiang (help·info), formerly romanized as C...                                                                                                                                                                                                                                                                                                                  
1   Zhejiang (help·info), formerly romanized as C...                                                                                     

In [8]:
paragraphs_df.loc[len(paragraphs_df.index)]=paragraphs_df.columns
para= paragraphs_df.iloc[:, 2]
para


0         Zhejiang (help·info), formerly romanized as C...
1         Zhejiang (help·info), formerly romanized as C...
2         Zhejiang (help·info), formerly romanized as C...
3         Zhejiang (help·info), formerly romanized as C...
4         Zhejiang (help·info), formerly romanized as C...
                               ...                        
75051    Although the two displayed great respect and a...
75052    On August 24, 1673, Dutch captain Anthonio Col...
75053    Eton has a long list of distinguished former p...
75054    After the Treaty of Utrecht (1713), Sicily was...
75055     Zhejiang (help·info), formerly romanized as C...
Name:  Zhejiang (help·info), formerly romanized as Chekiang, is an eastern coastal province of China. Zhejiang is bordered by Jiangsu province and Shanghai municipality to the north, Anhui province to the northwest, Jiangxi province to the west, and Fujian province to the south; to the east is the East China Sea, beyond which lie the Ryukyu Islands 

In [9]:
queries_df.loc[len(queries_df.index)]=queries_df.columns
queries_df.columns=['question', 'T/F', 'Answer']
queries_df.head()

,question,T/F,Answer
0,"If you need a date of four days less, what do ...",True,['-4']
1,What was Hanover's attitude toward Austria?,True,['-belligerent Hanover']
2,What is an example of a suffix used by Old Eng...,True,['-de']
3,The Modern English ending -'s is derived from ...,True,['-es']
4,How are the games broadcast now?,True,"[', FA Cup matches are shown live by ITV acros..."


In [10]:
queries_df['question']=queries_df['question'].fillna("")
para=para.fillna("")

In [11]:
# Initialize tokenizers and models
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
generator_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
generator = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequence

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Parnika Mittal\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Parnika Mittal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

c:\Users\Parnika Mittal\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
# # Function to encode a single paragraph and save the embedding
# def encode_and_save_paragraph(paragraph, index, save_path):
#     context_inputs = context_tokenizer(paragraph, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
#     with torch.no_grad():
#         context_embedding = context_encoder(**context_inputs).pooler_output
#     torch.save(context_embedding, f"{save_path}/embedding_{index}.pt")

# # Encode and save all paragraphs
# save_path = r'D:\ESC4\task4\paragraph_embeddings'
# os.makedirs(save_path, exist_ok=True)

# for i, paragraph in enumerate(para):
#     encode_and_save_paragraph(paragraph, i, save_path)
#     print(f"Encoded and saved paragraph {i + 1}/{len(paragraphs_df)}")

# Function to encode paragraphs in batches
# def encode_paragraphs(paragraphs, batch_size=500, max_length=512):
#     num_batches = (len(paragraphs) + batch_size - 1) // batch_size
#     embeddings = []

#     for i in range(num_batches):
#         batch_paragraphs = paragraphs[i * batch_size : (i + 1) * batch_size]
#         context_inputs = context_tokenizer(batch_paragraphs, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
#         with torch.no_grad():
#             batch_embeddings = context_encoder(**context_inputs).pooler_output
#         embeddings.append(batch_embeddings.cpu().numpy())
#         print(f"Encoded first {5*i}00 paragraphs")

#     return np.vstack(embeddings)

def encode_paragraphs(paragraphs):
    embeddings = model.encode(paragraphs, batch_size=32, convert_to_tensor=True, show_progress_bar=True)
    return embeddings

# Encode all paragraphs
paragraphs = para.tolist()
context_embeddings = encode_paragraphs(paragraphs)

Batches:   0%|          | 0/2346 [00:00<?, ?it/s]

In [13]:
# Function to retrieve top-K paragraphs using cosine similarity
def retrieve_top_k_paragraphs(query, k=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, context_embeddings)[0]
    top_k_scores, top_k_indices = torch.topk(cos_scores, k)
    top_k_paragraphs = [paragraphs[i] for i in top_k_indices]
    return top_k_paragraphs, top_k_indices

In [29]:
qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')

def get_answer(paragraph, question):
    inputs = tokenizer(question, paragraph, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = qa_model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
    return answer
# def generate_answer(question, context):
#     input_text = question + " " + context
#     generator_inputs = generator_tokenizer(input_text, return_tensors='pt', max_length=1024, truncation=True)
#     with torch.no_grad():
#         generated_outputs = generator.generate(**generator_inputs, max_length=50)
#     answer_start = torch.argmax(generated_outputs.start_logits)
#     answer_end = torch.argmax(generated_outputs.end_logits) + 1
#     answer = generator_tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
#     return answer

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def find_answer(query, top_paragraphs):
    for paragraph in top_paragraphs:
        answer = get_answer(paragraph, query)
        if answer:
            return paragraph, answer
    return None, "No answer found"

In [31]:
def predict_answer(query):
    top_k_paragraphs, top_k_indices = retrieve_top_k_paragraphs(query)
    # print(top_k_indices)
    answer_found = False
    answer_text = ""
    
    for paragraph in top_k_paragraphs:
        generated_text = get_answer(paragraph, query)
        if generated_text.strip():  # Simple heuristic to check if the model generated a non-empty response
            answer_found = True
            answer_text = generated_text
            break
    print(answer_text)

In [34]:
predict_answer("What happens if the soverign doesn't sign the bill within the six-month time frame?")

it becomes law without his signature


it becomes law without his signature


In [37]:
import gradio as gr

# def predict_ans(query):
#     ans=predict_answer(query)
#     return ans

# interface=gr.Interface(fn=predict_ans, inputs="text", outputs="text")
# interface.launch()

ans=gr.Interface(fn=predict_answer, inputs=gr.Textbox(), outputs=gr.Textbox())
ans.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.15.0, however version 4.29.0 is available, please upgrade.
--------
it becomes law without his signature


In [ ]:
def get_answer(paragraph, question):
    inputs = tokenizer(question, paragraph, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = qa_model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
    return answer

In [ ]:
def find_answer(query, top_paragraphs):
    for paragraph in top_paragraphs:
        answer = get_answer(paragraph, query)
        if answer:
            return paragraph, answer
    return None, "No answer found"

In [ ]:
for i in range(len(queries.index)-1):
    quer = queries.iloc[i]['query']
    top_5 = get_top_5_paragraphs(quer)
    answer_paragraph, answer = find_answer(quer, top_5)
    print("Paragraph:", answer_paragraph)
    print("Answer:", answer)

Paragraph: Up to February 28 in the calendar you are converting from add one day less or subtract one day more than the calculated value. Remember to give February the appropriate number of days for the calendar you are converting into. When you are subtracting days to move from Julian to Gregorian be careful, when calculating the Gregorian equivalent of February 29 (Julian), to remember that February 29 is discounted. Thus if the calculated value is -4 the Gregorian equivalent of this date is February 24.
Answer: one day more than the calculated value . remember to give february the appropriate number of days for the calendar you are converting into . when you are subtracting days to move from julian to gregorian be careful , when calculating the gregorian equivalent of february 29 ( julian ) , to remember that february 29 is discounted . thus if the calculated value is - 4
Paragraph: In 1837, the personal union of the United Kingdom and Hanover ended because William IV's heir in the 

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Paragraph: Promote and Publicize the Party's Agenda. An important aim of the minority leader is to develop an electorally attractive agenda of ideas and proposals that unites his or her own House members and that energizes and appeals to core electoral supporters as well as independents and swing voters. Despite the minority leader's restricted ability to set the House's agenda, there are still opportunities for him to raise minority priorities. For example, the minority leader may employ, or threaten to use, discharge petitions to try and bring minority priorities to the floor. If he or she is able to attract the required 218 signatures on a discharge petition by attracting majority party supporters, he or she can force minority initiatives to the floor over the opposition of the majority leadership. As a GOP minority leader once said, the challenges he confronted are to "keep our people together, and to look for votes on the other side."
Answer: the minority leader may employ , or th

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Paragraph: In 1845, Ireland was hit by a potato blight. In the next four years over a million Irish people died and another million emigrated in what became known as the Great Famine. In Ireland, Victoria was labelled "The Famine Queen". She personally donated £2,000 to famine relief, more than any other individual donor, and also supported the Maynooth Grant to a Roman Catholic seminary in Ireland, despite Protestant opposition. The story that she donated only £5 in aid to the Irish, and on the same day gave the same amount to Battersea Dogs Home, was a myth generated towards the end of the 19th century.
Answer: £2 , 000
Paragraph: In the nineteenth century Burke was praised by both liberals and conservatives. Burke's friend Philip Francis wrote that Burke "was a man who truly & prophetically foresaw all the consequences which would rise from the adoption of the French principles" but because Burke wrote with so much passion, people were doubtful of his arguments. William Windham spok

...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Paragraph: The United States Census Bureau estimates that the population of Florida was 20,271,272 on July 1, 2015, a 7.82% increase since the 2010 United States Census. The population of Florida in the 2010 census was 18,801,310. Florida was the seventh fastest-growing state in the U.S. in the 12-month period ending July 1, 2012. In 2010, the center of population of Florida was located between Fort Meade and Frostproof. The center of population has moved less than 5 miles (8 km) to the east and approximately 1 mile (1.6 km) to the north between 1980 and 2010 and has been located in Polk County since the 1960 census. The population exceeded 19.7 million by December 2014, surpassing the population of the state of New York for the first time.
Answer: 2010 united states census . the population of florida in the 2010 census was 18 , 801 , 310 . florida was the seventh fastest - growing state in the u . s . in the 12 - month period ending july 1 , 2012 . in 2010 , the center of population o

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Paragraph: Napoleon was born in Corsica to a relatively modest family of noble Tuscan ancestry. Napoleon supported the French Revolution from the outset in 1789 while serving in the French army, and he tried to spread its ideals to Corsica but was banished from the island in 1793. Two years later, he saved the French government from collapse by firing on the Parisian mobs with cannons. The Directory rewarded Napoleon by giving him command of the Army of Italy at age 26, when he began his first military campaign against the Austrians and their Italian allies, scoring a series of decisive victories that made him famous all across Europe. He followed the defeat of the Allies in Europe by commanding a military expedition to Egypt in 1798, invading and occupying the Ottoman province after defeating the Mamelukes and launching modern Egyptology through the discoveries made by his army.
Answer: 1793
Paragraph: In the nineteenth century Burke was praised by both liberals and conservatives. Bur

KeyboardInterrupt: 